# ETL
### Extracción, Transformación y Carga

In [5]:
import pandas as pd
import json
import re
import numpy as np
from textblob import TextBlob

# Extraccion de datos

### Extracción de los datos en formato.json

# 1-Reviews

Se carga el archivo 'australian_user_reviews.json', interpretamos sus líneas como objetos JSON, y luego los almacenamos en un DataFrame de Pandas para que puedan ser manipulados y analizados más fácilmente

In [7]:
input_file = 'raw data/australian_user_reviews.json'

# Leer el contenido del archivo JSON
with open(input_file, 'r', encoding='utf-8') as f:
    data = f.readlines()

# Convertir las líneas a registros JSON
records = [eval(line.strip()) for line in data]

# Crear el DataFrame a partir de los registros
df_reviews= pd.DataFrame(records)

Damos una mirada a los registros que contiene el df_reviews

In [ ]:
df_reviews.sample(5)

Encontramos una particularidad en los datos contenidos en la columna reviews asi que veremos su contenido mas detalladamente

In [ ]:
df_reviews['reviews'][0]

al parecer por cada registro en df_reviews['reviews'] encontramos una lista de diccionarios. Dado que la columna con la mayor cantidad de informacion de interes es ['reviews] procedemos a eliminar los valores duplicados y los nulos de esta columna si los llegase a haber.

In [8]:
df_reviews=df_reviews.dropna(subset=['reviews'])
df_df_reviews = df_reviews.drop_duplicates(subset=['reviews'])

# Items

Cargamos el archivo 'australian_users_items.json' de la misma forma que el archivo 'australian_user_reviews.json'

In [24]:
input_file = 'raw data/australian_users_items.json'

# Leer el contenido del archivo JSON
with open(input_file, 'r', encoding='utf-8') as f:
    data = f.readlines()

# Convertir las líneas a registros JSON
records = [eval(line.strip()) for line in data]

# Crear el DataFrame a partir de los registros
df_items= pd.DataFrame(records)

visualizamos los datos en el df_items

In [25]:
df_items.head(2)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


notamos cierto parecido en el formato de la columna ['reviews] en el archivo df_reviews y la columna ['items] de el archivo actual

In [14]:
df_items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

constatamos que ambas columnas contienen listas de diccionarios 

# Games

carga un archivo JSON , interpreta cada línea como un objeto JSON y luego almacena estos objetos en un DataFrame de Pandas para su posterior manipulación y análisis. 

In [2]:
data=[]
with open('raw data\output_steam_games.json', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
df_games=pd.DataFrame(data)

visualizamos que contiene nuestro dataframe

In [3]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Ya que notamos varios registros completamente llenos de valores nulos vamos a eliminarlos para luego hacer una visualizacion mas profunda del dataframe

In [30]:
df_games=df_games.dropna(how='all')
df_games.head(20)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
88315,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,http://store.steampowered.com/app/772540/Battl...,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",http://steamcommunity.com/app/772540/reviews/?...,"[Single-player, Steam Achievements]",3.99,False,772540,Trickjump Games Ltd
88316,NaN,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Basic Pass,SNOW - All Access Basic Pass,http://store.steampowered.com/app/774276/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774276/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",9.99,False,774276,Poppermost Productions
88317,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Pro Pass,SNOW - All Access Pro Pass,http://store.steampowered.com/app/774277/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774277/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",18.99,False,774277,Poppermost Productions
88318,Poppermost Productions,"[Free to Play, Indie, Simulation, Sports]",SNOW - All Access Legend Pass,SNOW - All Access Legend Pass,http://store.steampowered.com/app/774278/SNOW_...,2018-01-04,"[Free to Play, Indie, Simulation, Sports]",http://steamcommunity.com/app/774278/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",29.99,False,774278,Poppermost Productions
88319,RewindApp,"[Casual, Indie, Racing, Simulation]",Race,Race,http://store.steampowered.com/app/768800/Race/,2018-01-04,"[Indie, Casual, Simulation, Racing]",http://steamcommunity.com/app/768800/reviews/?...,"[Single-player, Multi-player, Partial Controll...",NaN,False,768800,RewindApp


Damos el proceso de extraccion de los datos por completado

# Transformacion de los datos

Empezamos el proceso de transformacion de los datos para prepararlos para su analisis y almacenamiento

debido a la naturaleza de los datos almacenados en las columnas ['reviews'] y ['items'] crearemos una funcion encargada de desanidarlas llamada data_processor, cuya idea principal es la de tomar cada diccionario en la columna y añadirle los datos correspondientes al usuario para asi lograr que cada diccionario contenga informacion del usuario del que proviene y agrupar estos diccionarios en una lista que luego convertira en un dataframe.

In [4]:
# Definición de la función 'data_processor' que transforma un DataFrame 'df'.
# Esta función toma 'df', una lista de 'columnas' y una 'columna_target' como entrada.
#la columna target hae alucion a la columna que contenga los datos anidados
def data_processor(df, columnas, columna_target):
    
    # Inicializa una lista vacía llamada 'lista_dics_datos' para almacenar los datos transformados.
    lista_dics_datos = []
    
    # Itera a través de las filas del DataFrame 'df'.
    for _, row in df.iterrows():
        
        # Extrae el 'user_id' de la fila actual.
        user_id = row['user_id']
        
        # Inicializa las variables 'steam_id' y 'items_count' con valores predeterminados.
        steam_id = 'steam_id'
        items_count = 'items_count'
        
        # Extrae el 'user_url' de la fila actual.
        user_url = row['user_url']
        
        # Inicializa la variable 'items' en 'False'.
        items = False
        
        # Verifica si las columnas 'steam_id' y 'items_count' están en el DataFrame 'df'(para diferenciar cual de los dos dfs esta procesando).
        # Si están presentes, actualiza 'steam_id' y 'items_count' y establece 'items' en 'True' que confirma que se trata del dataframe df_items.
        if steam_id in df.columns and items_count in df.columns:
            steam_id = row['steam_id']
            items_count = row['items_count']
            items = True 
            
        # Itera a través de los elementos en la columna especificada por 'columna_target'.
        for elemento in row[columna_target]:
            
            # Crea un nuevo diccionario 'elemento_data' que contiene la información de la fila actual.
            elemento_data = elemento.copy()
            
            # Agrega información adicional al diccionario 'elemento_data'.
            elemento_data['user_id'] = user_id
            elemento_data['user_url'] = user_url
            
            # Si 'items' es 'True', agrega 'steam_id' y 'items_count' al diccionario 'elemento_data'.
            if items:
                elemento_data['steam_id'] = steam_id
                elemento_data['items_count'] = items_count
                      
            # Agrega el diccionario 'review_data' a la lista 'listadatos'.
            lista_dics_datos.append(elemento_data)

    # Crea un nuevo DataFrame 'df_limpio' a partir de la lista 'lista_dics_datos'
    # utilizando las columnas especificadas en 'columnas'.
    df_limpio = pd.DataFrame(lista_dics_datos, columns=columnas)
    
    # Devuelve el nuevo DataFrame 'df_limpio'.
    return df_limpio



# Reviews

pasamos el df_reviews por la funcion data_processor y definimos las columnas que tendra el dataframe que esperamos

In [9]:
columnas_review = ['user_id','user_url', 'item_id', 'funny', 'posted', 'last_edited', 'helpful', 'recommend', 'review']
df_reviews_limpio=data_processor(df_reviews,columnas_review,'reviews')

visualizamos el contenido de nuestro nuevo dataframe

In [ ]:
df_reviews_limpio.head()

vemos que ahora en la columna review tenemos opiniones escritas de los usuarios para los juegos. Pero para una interpretacion mas general del modelo lo ideal seria tener valoraciones en un formato numerico y para esto podriamos pasar cada review  por un análisis de sentimiento con NLP y le otrogamos la siguiente escala '0' si es malo, '1' si es neutral y '2' si es positivo

In [10]:
# Definición de una función llamada 'analisis_sentimiento' que analiza el sentimiento de un texto dado.
def analisis_sentimiento(text):
    
    # Verifica si 'text' es una cadena de texto (str).
    if isinstance(text, str):  
        
        # Crea una instancia de la clase TextBlob a partir del texto proporcionado.
        analysis = TextBlob(text)
        
        # Calcula la polaridad del sentimiento del texto utilizando TextBlob.
        sentiment = analysis.sentiment.polarity
        
        # Compara la polaridad del sentimiento calculada.
        # Si es mayor que 0, devuelve 2 (positivo).
        if sentiment > 0:
            return 2
        # Si es menor que 0, devuelve 0 (negativo).
        elif sentiment < 0:
            return 0
        # Si es igual a 0, devuelve 1 (neutro).
        else:
            return 1
    else:
        # Si 'text' no es una cadena de texto, devuelve 1 (neutro).
        return 1

In [ ]:
#aplicamos la funcion analisis de sentimiento a la columna review y mostramos el nuevo dataframe
df_reviews_limpio['review']=df_reviews_limpio['review'].apply(analisis_sentimiento)
df_reviews_limpio.head()

Para la columna posted vemos un formato de fecha que me gustaria confirmar que fuera uniforme.

In [ ]:
df_reviews_limpio['posted'].sample(20)

vemos que en  su mayoria mantienen un formato posted %B %d, %Y. y los que no lo cumplen parecen corresponder al año en el que fueron tomados los datos pero debido a que desconocemos el año donde fueron tomados estos datos no nos aportan ninguna informacion. por lo cual mi funcion devolvera nan para los valores que no coincidan con el primer formato 

In [18]:
#elimino la palabra Posted de todos los registros
df_reviews_limpio['posted']=df_reviews_limpio['posted'].str.replace('Posted ', '')

In [19]:
#defino la funcion que convierte a formato fecha los datos de la columna 
def convert_date(text):
    try:
        return pd.to_datetime(text, format='%B %d, %Y.', errors='raise')
    except ValueError:
        return np.nan

Aplicamos la funcion para convertir los datos a fechas 

In [20]:
df_reviews_limpio['date'] = df_reviews_limpio['posted'].apply(convert_date)

Pasamos los valores booleanos de la columna recommend a enteros 

In [21]:
df_reviews_limpio['recommend']= df_reviews_limpio['recommend'].astype(int)

le doy forma al dataframe limpio y elimino las columnas que no pienso utilizar

In [22]:
df_reviews_limpio=df_reviews_limpio[['user_id', 'user_url','item_id', 'recommend','review', 'date' ]]

# Items

paso el df_items por la funcion data_processor para obtener el df_items_limpio y eliminamos los registros duplicados en caso de haberlos 

In [26]:
lista_col=['user_id','user_url','items_count','steam_id','item_id','item_name','playtime_forever','playtime_2weeks']

df_items_limpio = data_processor(df_items,lista_col,'items')
df_items_limpio = df_items_limpio.drop_duplicates()


paso todos los nombres de los juegos a minusculas para mantener una uniformidad

In [ ]:
df_items_limpio['item_name'] = df_items_limpio['item_name'].str.lower()
df_items_limpio.head(2)

# Games

In [31]:
df_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL


vemos que las columnas app_name-title y genres-tags compartes practicamente los mismos registros, sabiendo que la columna id sera la base de nuestro sistema de busqueda dentro de este dataframe (es decir si un juego no tiene id practicamente sera descartado) queremos saber cual de las columnas duplicadas es mas completa respecto a la columna id 

In [35]:
print(f'La columna app_name contiene { len(df_games["app_name"].dropna()) } valores no nulos')
print(f'La columna title contiene {len(df_games["title"].dropna())} valores no nulos')
print(f'La columna id contiene {len(df_games["id"].dropna())} valores no nulos\n')


print(f'La columna genres contiene { len(df_games["genres"].dropna()) } valores no nulos')
print(f'La columna tags contiene {len(df_games["tags"].dropna())} valores no nulos')
print(f'La columna id contiene {len(df_games["id"].dropna())} valores no nulos\n')

La columna app_name contiene 32133 valores no nulos
La columna title contiene 30085 valores no nulos
La columna id contiene 32133 valores no nulos

La columna genres contiene 28852 valores no nulos
La columna tags contiene 31972 valores no nulos
La columna id contiene 32133 valores no nulos



vemos que las columnas mas completas son app_name y  tags por lo que descartaremos las otras dos en el dataframe final

In [36]:
df_games=df_games[['developer','app_name','release_date','id','tags','price']].dropna(subset='id')
#renombramos la columna tags para que sea un poco mas claro que su contenido especifica el genero al que pertenece el juego
df_games=df_games.rename(columns={'tags':'genres'})

eliminamos posibles espacios invisibles en la columna app_name y pasamos los registros en las columnas app_name y developer  a minusculas para mantener la uniformidad

In [37]:
df_games['app_name']=df_games['app_name'].astype(str).str.strip()
df_games['app_name']=df_games['app_name'].str.lower()
df_games['developer']=df_games['developer'].str.lower()

reviso el contenido de la columna price

In [ ]:
df_games['price'].sample(25)

al ver que la mayoria de los registros son numericos o contienen la palabra free para decir que no tiene ningun costo me gustaria ver los casos donde no hay ni numeros ni palabras para lo cual creare una funcion 

In [481]:
# Función para verificar si una cadena no contiene números ni la palabra "Free"
def no_contiene_numeros_ni_palabra_libre(cadena):
    # Convierte 'cadena' en una cadena de texto (str) para asegurarse de que se pueda aplicar el siguiente código.
    cadena = str(cadena)
    
    # Si la cadena es 'nan' (valor especial que representa un dato faltante o nulo), devuelve False.
    if cadena == 'nan':
        return False  # No considerar NaN
    
    # Utiliza expresiones regulares para buscar números (\d) o la palabra "Free" (insensible a mayúsculas/minúsculas \bFree\b).
    # La función 're.search' devuelve un objeto que indica si se encontró una coincidencia.
    # 'flags=re.I' hace que la búsqueda de la palabra "Free" sea insensible a mayúsculas y minúsculas.
    return not (re.search(r'\d', cadena) or re.search(r'\bFree\b', cadena, flags=re.I))

# Filtrar y obtener valores que no contienen números ni la palabra "Free" en la columna 'price' del DataFrame 'df_games'.
filtro = df_games['price'].apply(no_contiene_numeros_ni_palabra_libre)
valores_sin_numeros_ni_palabra_libre = df_games.loc[filtro, 'price']

# Imprimir valores sin números ni la palabra "Free"
print("Valores sin números ni la palabra 'Free':")
print(valores_sin_numeros_ni_palabra_libre)

Valores sin números ni la palabra 'Free':
90715                       Install Now
91181     Play WARMACHINE: Tactics Demo
92142                     Install Theme
92228                       Third-party
92336                          Play Now
111044                    Play the Demo
114527                         Play Now
120148                      Third-party
Name: price, dtype: object


podemos ver que incluso los valores que no contienen la palabra free podrian considerarse como contenido gratuito. Defino contenido gratuito como contenido descargable sin necesidad de realizar una transaccion monetaria. sabiendo esto creare una funcion que me devuelva solo los numeros contenidos en la cadena de texto en la columna y todas las que no tengan numeros las convierta en 0 porque se tomara como gratuito 

In [482]:
# Definición de una función llamada 'extraer_numeros' que toma una cadena de texto como entrada.
def extraer_numeros(cadena):
    # Utiliza expresiones regulares para encontrar todos los números en la cadena.
    # El patrón \d+\.\d+|\d+ busca números enteros o decimales en la cadena.
    numeros_encontrados = re.findall(r'\d+\.\d+|\d+', str(cadena))
    
    # Si se encuentran números en la cadena:
    if numeros_encontrados:
        # Une los números encontrados utilizando comas y devuelve la cadena resultante.
        return ','.join(numeros_encontrados)
    
    # Si no se encuentran números en la cadena, devuelve 0.
    return 0

# Aplicar la función a la columna 'price' para obtener los números
df_games['price'] = df_games['price'].apply(extraer_numeros)

revisamos una vez mas el contenido de la columna price

In [ ]:
df_games['price'].sample(20)

como ya hemos venido haciendo me gustaria volver todos los valores dentro de la columna genres a minusculas para lo cual creare una funcion simple y se la aplicare a la columna

In [514]:
def convertir_lista_a_minusculas(lista):
    if isinstance(lista, list):
        return [texto.lower() for texto in lista]
    return []

In [515]:
df_games['genres'] =df_games['genres'].apply(convertir_lista_a_minusculas)
df_games.head(2)

,developer,app_name,release_date,id,genres,price
88310,kotoshiro,lost summoner kitty,2018-01-04,761140,"[strategy, action, indie, casual, simulation]",4.99
88311,secret level srl,ironbound,2018-01-04,643980,"[free to play, strategy, indie, rpg, card game...",0


# Cargamos los datos 

In [516]:
df_reviews_limpio.to_csv('dataset/df_reviews_limpio.csv', index=False)

In [517]:
df_items_limpio.to_csv('dataset/df_items_limpio.csv', index=False)

In [518]:
df_games.to_csv('dataset/df_games_limpio.csv', index=False)